In [ ]:
using Flux
using Flux.Data: DataLoader
using BSON: @save, @load # for data saving
using ProgressMeter
using LinearAlgebra
using LaTeXStrings
using Plots; gr()
using MAT

In [ ]:
file = matopen("../greenlearning/examples/datasets/laplace.mat")
varnames = collect(names(file))
for varname in varnames
    val = read(file, varname)
    # eval(parse("$varname=$val"))
    @eval $(Symbol(varname)) = $val
end
close(file)
println(varnames)
X = reshape(X, 100)
Y = reshape(Y, 200);

In [ ]:
# number of data: 100
p1 = begin
    plot(xlim=(0,1), ylim=(-1,1), title=L"u(x)")
    plot!(X, U, label="", c="gray", alpha=0.2)
    plot!(X, U_hom, label="homogeneous", lw=5) 
end
p2 = begin
    plot(xlim=(0,1), title=L"f(x)")
    plot!(Y, F, label="", c="gray", alpha=0.2)
end
plot(p1, p2, size=(1200,400))

In [ ]:
hidden_size = 50
G_network = Chain(
    Dense(2, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, 1)
)
U_hom_network = Chain(
    Dense(1, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, hidden_size, relu),
    Dense(hidden_size, 1)
)
parameters = Flux.params(G_network, U_hom_network);

In [ ]:
batch_size = 20
train_loader = DataLoader((U, F), batchsize=batch_size, shuffle=true);

In [ ]:
function loss(U_train, F_train)
    l = 0.0
    U_hom_predict = [U_hom_network([x])[1] for x in X]
    # G_predict = [G_network([x,y])[1] for x in X, y in Y]
    G_predict = [[G_network([x,y])[1] for x in X] for y in Y]
    G_predict = hcat(G_predict...)
    num = size(U_train, 2)
    for n in 1:num
        @views u = U_train[:,n] # 100×1
        @views f = F_train[:,n] # 200×1
        Gf = dy*G_predict*(trapezoidal_y.*f)
        integ = (u-U_hom_predict-Gf).^2
        loss_u = dx*dot(integ, trapezoidal_x)
        u_L2 = dx*dot(u.^2, trapezoidal_x)
        l += loss_u/u_L2
    end
    return l/num
end

In [ ]:
dx = X[2]-X[1]
dy = Y[2]-Y[1]
Nu = length(X)
Nf = length(Y)
trapezoidal_x = ones(Nu)
trapezoidal_x[begin] = 0.5
trapezoidal_x[end] = 0.5
trapezoidal_y = ones(Nf)
trapezoidal_y[begin] = 0.5
trapezoidal_y[end] = 0.5

In [ ]:
epochs = 20

In [ ]:
@showprogress for epoch in 1:epochs
    for (U_train, F_train) in train_loader
        gs = gradient(()->loss(U_train, F_train), parameters)
        Flux.Optimise.update!(opt, parameters, gs)
    end
end

In [ ]:
xs = 0:0.01:1
G_kernel = [G_network([x,y])[1] for x in xs, y in xs]
p1 = heatmap(xs, xs, G_kernel, title="Learned green function")

function laplace_exact_kernel(x,y)
    if x>y
        return y*(1-x)
    else
        return x*(1-y)
    end
end
G_exact = [laplace_exact_kernel(x,y) for x in xs, y in xs]
p2 = heatmap(xs, xs, G_exact, title="Exact green function")
plot(p1, p2, size=(1200,400))

In [ ]:
plot(p1, p2, size=(1200,400))
savefig("laplace.png")

In [ ]:
@save "test.bson" G_network U_hom_network opt

In [ ]:
@load "laplace.bson" G_network U_hom_network opt